In [1]:
try:
    __IPYTHON__
    USING_IPYTHON = True
    %load_ext autoreload
    %autoreload 2
except NameError:
    USING_IPYTHON = False

#### Argparse

In [2]:
import argparse
ap = argparse.ArgumentParser()
ap.add_argument('project_root', help='')
ap.add_argument('--mrp-data-dir', default='data', help='')
ap.add_argument('--mrp-test-dir', default='src/tests', help='')
ap.add_argument('--allennlp-serialization-dir-template', default='output/allennlp-serialization-2019-07-21-{}', help='')
ap.add_argument('--tests-fixtures-file-template', default='fixtures/{}-test.jsonl', help='')

ap.add_argument('--graphviz-sub-dir', default='visualization/graphviz', help='')
ap.add_argument('--train-sub-dir', default='training', help='')
ap.add_argument('--companion-sub-dir', default='companion')
ap.add_argument('--jamr-alignment-file', default='jamr.mrp')

ap.add_argument('--evaluation-allennlp-mrp-json-file', default='evaluation/allennlp-evaluation.mrp')
ap.add_argument('--test-input-file', default='evaluation/input.mrp', help='')
ap.add_argument('--test-companion-file', default='evaluation/udpipe.mrp', help='')
ap.add_argument('--allennlp-cache-dir', default='allennlp-cache', help='')
ap.add_argument('--allennlp-mrp-json-file-template', default='allennlp-mrp-json-big-{}-{}.jsonl', help='')
ap.add_argument('--data-size-limit', type=int, default=10000, help='')

ap.add_argument('--mrp-file-extension', default='.mrp')
ap.add_argument('--companion-file-extension', default='.conllu')
ap.add_argument('--graphviz-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png')
ap.add_argument('--parse-plot-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.png')

ap.add_argument('--cuda-device', type=int, default=0)

arg_string = """
    /data/proj29_ds1/home/slai/mrp2019
"""
arguments = [arg for arg_line in arg_string.split(r'\\n') for arg in arg_line.split()]

In [3]:
if USING_IPYTHON:
    args = ap.parse_args(arguments)
else:
    args = ap.parse_args()

In [4]:
args

Namespace(allennlp_cache_dir='allennlp-cache', allennlp_mrp_json_file_template='allennlp-mrp-json-big-{}-{}.jsonl', allennlp_serialization_dir_template='output/allennlp-serialization-2019-07-21-{}', companion_file_extension='.conllu', companion_sub_dir='companion', cuda_device=0, data_size_limit=10000, evaluation_allennlp_mrp_json_file='evaluation/allennlp-evaluation.mrp', graphviz_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png', graphviz_sub_dir='visualization/graphviz', jamr_alignment_file='jamr.mrp', mrp_data_dir='data', mrp_file_extension='.mrp', mrp_test_dir='src/tests', parse_plot_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.png', project_root='/data/proj29_ds1/home/slai/mrp2019', test_companion_file='evaluation/udpipe.mrp', test_input_file='evaluation/input.mrp', tests_fixtures_file_template='fixtures/{}-test.jsonl', train_sub_dir='training')

#### Library imports

In [5]:
import json
import logging
import os
import pprint
import re
import string
from collections import Counter, defaultdict, deque

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import plot_util
import torch
from action_state import mrp_json2parser_states, _generate_parser_action_states
from action_state import ERROR, APPEND, RESOLVE, IGNORE
from preprocessing import (CompanionParseDataset, MrpDataset, JamrAlignmentDataset,
                           read_companion_parse_json_file, read_mrp_json_file, parse2parse_json)            
from torch import nn
from tqdm import tqdm

#### ipython notebook specific imports

In [6]:
if USING_IPYTHON:
    # matplotlib config
    %matplotlib inline

In [7]:
sh = logging.StreamHandler()
formatter = logging.Formatter('%(levelname)-8s [%(name)s:%(lineno)d] %(message)s')
sh.setFormatter(formatter)
logging.basicConfig(
    level=logging.DEBUG, 
    handlers=[sh]
)
mute_logger_names = ['allennlp.data.iterators.data_iterator']
for logger_name in mute_logger_names:
    logger = logging.getLogger(logger_name)  # pylint: disable=invalid-name
    logger.setLevel(logging.INFO)

logger = logging.getLogger(__name__)  # pylint: disable=invalid-name
logger.setLevel(logging.INFO)

### Constants

In [8]:
UNKWOWN = 'UNKWOWN'

### Load data

In [9]:
train_dir = os.path.join(args.project_root, args.mrp_data_dir, args.train_sub_dir)

In [10]:
mrp_dataset = MrpDataset()

In [11]:
frameworks, framework2dataset2mrp_jsons = mrp_dataset.load_mrp_json_dir(
    train_dir, args.mrp_file_extension)

frameworks: 100%|██████████| 5/5 [00:18<00:00,  3.15s/it]t/s]


In [12]:
framework2dataset2mrp_jsons.keys()

dict_keys(['ucca', 'psd', 'eds', 'dm', 'amr'])

### Data Preprocessing companion

In [13]:
companion_dir = os.path.join(args.project_root, args.mrp_data_dir, args.companion_sub_dir)

In [14]:
cparse_dataset = CompanionParseDataset()

In [15]:
dataset2cid2parse = cparse_dataset.load_companion_parse_dir(companion_dir, args.companion_file_extension)

INFO     [preprocessing:179] framework amr found
dataset: 100%|██████████| 13/13 [00:01<00:00,  9.77it/s]
INFO     [preprocessing:179] framework dm found
dataset: 100%|██████████| 5/5 [00:04<00:00,  1.03s/it]
INFO     [preprocessing:179] framework ucca found
dataset: 100%|██████████| 6/6 [00:00<00:00, 24.30it/s]


In [16]:
dataset2cid2parse_json = cparse_dataset.convert_parse2parse_json()

In [17]:
dataset2cid2parse.keys()

dict_keys(['amr-guidelines', 'bolt', 'cctv', 'dfa', 'dfb', 'fables', 'lorelei', 'mt09sdl', 'proxy', 'rte', 'wb', 'wiki', 'xinhua', 'wsj', 'ewt'])

In [18]:
# Some data is missing
'20003001' in dataset2cid2parse['wsj']

False

### Load JAMR alignment data

In [19]:
jalignment_dataset = JamrAlignmentDataset()

In [20]:
cid2alignment = jalignment_dataset.load_jamr_alignment_file(os.path.join(
    args.project_root,
    args.mrp_data_dir,
    args.companion_sub_dir,
    args.jamr_alignment_file
))

### Load testing data

In [21]:
test_input_filename = os.path.join(args.project_root, args.mrp_data_dir, args.test_input_file)
test_companion_filename = os.path.join(args.project_root, args.mrp_data_dir, args.test_companion_file)

In [22]:
test_mrp_jsons = read_mrp_json_file(test_input_filename)
test_parse_jsons = read_companion_parse_json_file(test_companion_filename)

In [23]:
parse_json = test_parse_jsons['102990']

In [24]:
mrp_json = framework2dataset2mrp_jsons['psd']['wsj'][1]

In [25]:
test_configs = [
    ('ucca', 'wiki', 70),
]
framework, dataset, idx = test_configs[0]

In [26]:
mrp_json = framework2dataset2mrp_jsons[framework][dataset][idx]
cid = mrp_json.get('id')

In [27]:
parse_json = dataset2cid2parse_json[dataset][cid]

In [28]:
doc = mrp_json['input']

In [29]:
doc

'In the final minute of the game, Johnson had the ball stolen by Celtics center Robert Parish, and then missed two free throws that could have won the game.'

In [30]:
token_pos = 0
anchors = []
char_pos2tokenized_parse_node_id = []

for node_id, node in enumerate(parse_json.get('nodes')):
    label = node.get('label')
    label_size = len(label)
    while doc[token_pos] == ' ':
        token_pos += 1
        char_pos2tokenized_parse_node_id.append(node_id)
    anchors.append((token_pos, token_pos + label_size))
    char_pos2tokenized_parse_node_id.extend([node_id] * (label_size))
    print(node_id, doc[token_pos: token_pos + label_size], anchors[-1], len(char_pos2tokenized_parse_node_id))
    token_pos += label_size

0 In (0, 2) 2
1 the (3, 6) 6
2 final (7, 12) 12
3 minute (13, 19) 19
4 of (20, 22) 22
5 the (23, 26) 26
6 game (27, 31) 31
7 , (31, 32) 32
8 Johnson (33, 40) 40
9 had (41, 44) 44
10 the (45, 48) 48
11 ball (49, 53) 53
12 stolen (54, 60) 60
13 by (61, 63) 63
14 Celtics (64, 71) 71
15 center (72, 78) 78
16 Robert (79, 85) 85
17 Parish (86, 92) 92
18 , (92, 93) 93
19 and (94, 97) 97
20 then (98, 102) 102
21 missed (103, 109) 109
22 two (110, 113) 113
23 free (114, 118) 118
24 throws (119, 125) 125
25 that (126, 130) 130
26 could (131, 136) 136
27 have (137, 141) 141
28 won (142, 145) 145
29 the (146, 149) 149
30 game (150, 154) 154
31 . (154, 155) 155


In [31]:
doc

'In the final minute of the game, Johnson had the ball stolen by Celtics center Robert Parish, and then missed two free throws that could have won the game.'

In [32]:
len(char_pos2tokenized_parse_node_id)

155

In [33]:
doc = mrp_json['input']

In [34]:
mrp_json['tops']

[34]

In [35]:
mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
    mrp_json, 
    tokenized_parse_nodes=parse_json['nodes'],
)

In [36]:
(
    doc,
    nodes,
    node_id2node,
    edge_id2edge,
    top_oriented_edges,
    token_nodes,
    # abstract_node_id_set,
    parent_id2indegree,
    # parent_id2child_id_set,
    # child_id2parent_id_set,
    # child_id2edge_id_set,
    # parent_id2edge_id_set,
    # parent_child_id2edge_id_set,
    parse_nodes_anchors,
    char_pos2tokenized_node_id,
    curr_node_ids,
    token_states,
    actions,
) = mrp_meta_data

In [37]:
curr_node_ids = mrp_meta_data[-3]
token_states = mrp_meta_data[-2]
actions = mrp_meta_data[-1]

In [38]:
*_, curr_node_ids, token_states, actions = mrp_meta_data

In [39]:
actions[1]

(1,
 (1,
  0,
  {'id': 0,
   'anchors': [{'from': 0, 'to': 2}],
   'label': 'In',
   'propagate_label': 'R'},
  [[]]))

In [40]:
actions[-4:]

[(1,
  (3,
   2,
   {'id': 41, 'propagate_label': 'A'},
   [[{'source': 41,
      'target': 28,
      'label': 'E',
      'id': 15,
      'parent': 41,
      'child': 28}],
    [{'source': 41,
      'target': 29,
      'label': 'C',
      'id': 24,
      'parent': 41,
      'child': 29}],
    [{'source': 41,
      'target': 30,
      'label': 'U',
      'id': 17,
      'parent': 41,
      'child': 30}]])),
 (1,
  (4,
   3,
   {'id': 40, 'propagate_label': 'H'},
   [[{'source': 40,
      'target': 25,
      'label': 'D',
      'id': 40,
      'parent': 40,
      'child': 25}],
    [{'source': 40,
      'target': 26,
      'label': 'F',
      'id': 32,
      'parent': 40,
      'child': 26}],
    [{'source': 40,
      'target': 27,
      'label': 'P',
      'id': 2,
      'parent': 40,
      'child': 27}],
    [{'source': 40,
      'target': 41,
      'label': 'A',
      'id': 16,
      'parent': 40,
      'child': 41}]])),
 (1,
  (3,
   2,
   {'id': 39, 'propagate_label': 'H'},
   [[{'s

In [41]:
for curr_node_id, action, token_state in zip(curr_node_ids, actions, token_states):
    action_type, params = action
#     pprint.pprint((curr_node_id, action[0]))
#     pprint.pprint(([token_group[:4] for token_group in token_state]))
    pprint.pprint((curr_node_id, action[0], [token_group[:5] for token_group in token_state]))

(0, 0, [(0, False, 'In', [])])
(1, 1, [(0, True, 'R', [(0, False, 'In', [])])])
(1, 0, [(0, True, 'R', [(0, False, 'In', [])]), (1, False, 'the', [])])
(2,
 1,
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])])])
(2,
 0,
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, False, 'final', [])])
(3,
 1,
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, True, 'E', [(2, False, 'final', [])])])
(3,
 0,
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, True, 'E', [(2, False, 'final', [])]),
  (3, False, 'minute', [])])
(4,
 1,
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, True, 'E', [(2, False, 'final', [])]),
  (3, True, 'C', [(3, False, 'minute', [])])])
(4,
 1,
 [(32,
   True,
   'E',
   [(0, True, 'R', [(0, False, 'In', [])]),
    (1, True, 'E', [(1, False, 'the', [])]),
    (2, True, 'E', [(

   True,
   'H',
   [(33,
     True,
     'T',
     [(32,
       True,
       'E',
       [(0, True, 'R', [(0, False, 'In', [])]),
        (1, True, 'E', [(1, False, 'the', [])]),
        (2, True, 'E', [(2, False, 'final', [])]),
        (3, True, 'C', [(3, False, 'minute', [])])]),
      (4, True, 'R', [(4, False, 'of', [])]),
      (5, True, 'E', [(5, False, 'the', [])]),
      (6, True, 'C', [(6, False, 'game', [])])]),
    (7, True, 'U', [(7, False, ',', [])]),
    (8, True, 'A', [(8, False, 'Johnson', [])]),
    (9, True, 'F', [(9, False, 'had', [])]),
    (34,
     True,
     'A',
     [(10, True, 'E', [(10, False, 'the', [])]),
      (11, True, 'C', [(11, False, 'ball', [])])]),
    (12, True, 'P', [(12, False, 'stolen', [])]),
    (36,
     True,
     'A',
     [(13, True, 'R', [(13, False, 'by', [])]),
      (35,
       True,
       'E',
       [(14, True, 'A', [(14, False, 'Celtics', [])]),
        (15, True, 'S', [(15, False, 'center', [])])]),
      (16, True, 'C', [(16, F

In [42]:
for curr_node_id, action, token_state in zip(curr_node_ids, actions, [[]] + token_states):
    action_type, params = action
    pprint.pprint((curr_node_id, action[0], (action[1] or [None])[:2], [token_group[:4] for token_group in token_state]))
    

(0, 0, [None], [])
(1, 1, (1, 0), [(0, False, 'In', [])])
(1, 0, [None], [(0, True, 'R', [(0, False, 'In', [])])])
(2, 1, (1, 0), [(0, True, 'R', [(0, False, 'In', [])]), (1, False, 'the', [])])
(2,
 0,
 [None],
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])])])
(3,
 1,
 (1, 0),
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, False, 'final', [])])
(3,
 0,
 [None],
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, True, 'E', [(2, False, 'final', [])])])
(4,
 1,
 (1, 0),
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, True, 'E', [(2, False, 'final', [])]),
  (3, False, 'minute', [])])
(4,
 1,
 (4, 3),
 [(0, True, 'R', [(0, False, 'In', [])]),
  (1, True, 'E', [(1, False, 'the', [])]),
  (2, True, 'E', [(2, False, 'final', [])]),
  (3, True, 'C', [(3, False, 'minute', [])])])
(4,
 0,
 [None],
 [(32,
   True,
   'E',
   [(0, True,

      (16, True, 'C', [(16, False, 'Robert', [])])])]),
  (18, True, 'U', [(18, False, ',', [])]),
  (19, True, 'L', [(19, False, 'and', [])]),
  (20, True, 'L', [(20, False, 'then', [])]),
  (21, True, 'D', [(21, False, 'missed', [])]),
  (22, True, 'D', [(22, False, 'two', [])]),
  (23, True, 'D', [(23, False, 'free', [])]),
  (24, True, 'P', [(24, False, 'throws', [])])])
(25,
 0,
 [None],
 [(37,
   True,
   'H',
   [(33,
     True,
     'T',
     [(32,
       True,
       'E',
       [(0, True, 'R', [(0, False, 'In', [])]),
        (1, True, 'E', [(1, False, 'the', [])]),
        (2, True, 'E', [(2, False, 'final', [])]),
        (3, True, 'C', [(3, False, 'minute', [])])]),
      (4, True, 'R', [(4, False, 'of', [])]),
      (5, True, 'E', [(5, False, 'the', [])]),
      (6, True, 'C', [(6, False, 'game', [])])]),
    (7, True, 'U', [(7, False, ',', [])]),
    (8, True, 'A', [(8, False, 'Johnson', [])]),
    (9, True, 'F', [(9, False, 'had', [])]),
    (34,
     True,
     'A',
  

In [43]:
actions

[(0, None),
 (1,
  (1,
   0,
   {'id': 0,
    'anchors': [{'from': 0, 'to': 2}],
    'label': 'In',
    'propagate_label': 'R'},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 1,
    'anchors': [{'from': 3, 'to': 6}],
    'label': 'the',
    'propagate_label': 'E'},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 2,
    'anchors': [{'from': 7, 'to': 12}],
    'label': 'final',
    'propagate_label': 'E'},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 3,
    'anchors': [{'from': 13, 'to': 19}],
    'label': 'minute',
    'propagate_label': 'C'},
   [[]])),
 (1,
  (4,
   3,
   {'id': 31, 'propagate_label': 'E'},
   [[{'source': 31,
      'target': 0,
      'label': 'R',
      'id': 31,
      'parent': 31,
      'child': 0}],
    [{'source': 31,
      'target': 1,
      'label': 'E',
      'id': 28,
      'parent': 31,
      'child': 1}],
    [{'source': 31,
      'target': 2,
      'label': 'E',
      'id': 21,
      'parent': 31,
      'child': 2}],
    [{'source': 31,
      't

In [44]:
token_states[1]

[(0, True, 'R', [(0, False, 'In', [])])]

In [45]:
[n['label'] for n in parse_json['nodes']]

['In',
 'the',
 'final',
 'minute',
 'of',
 'the',
 'game',
 ',',
 'Johnson',
 'had',
 'the',
 'ball',
 'stolen',
 'by',
 'Celtics',
 'center',
 'Robert',
 'Parish',
 ',',
 'and',
 'then',
 'missed',
 'two',
 'free',
 'throws',
 'that',
 'could',
 'have',
 'won',
 'the',
 'game',
 '.']

In [46]:
token_states[-1]

[(42,
  True,
  '<UCCA-TOP-NODE>',
  [(37,
    True,
    'H',
    [(33,
      True,
      'T',
      [(32,
        True,
        'E',
        [(0, True, 'R', [(0, False, 'In', [])]),
         (1, True, 'E', [(1, False, 'the', [])]),
         (2, True, 'E', [(2, False, 'final', [])]),
         (3, True, 'C', [(3, False, 'minute', [])])]),
       (4, True, 'R', [(4, False, 'of', [])]),
       (5, True, 'E', [(5, False, 'the', [])]),
       (6, True, 'C', [(6, False, 'game', [])])]),
     (7, True, 'U', [(7, False, ',', [])]),
     (8, True, 'A', [(8, False, 'Johnson', [])]),
     (9, True, 'F', [(9, False, 'had', [])]),
     (34,
      True,
      'A',
      [(10, True, 'E', [(10, False, 'the', [])]),
       (11, True, 'C', [(11, False, 'ball', [])])]),
     (12, True, 'P', [(12, False, 'stolen', [])]),
     (36,
      True,
      'A',
      [(13, True, 'R', [(13, False, 'by', [])]),
       (35,
        True,
        'E',
        [(14, True, 'A', [(14, False, 'Celtics', [])]),
         (

In [47]:
companion_parser_states, companion_meta_data = mrp_json2parser_states(
    parse_json,
    mrp_doc=doc,
    tokenized_parse_nodes=parse_json['nodes'],
)

In [48]:
logger.info(args.graphviz_file_template.format(
    framework, dataset, cid))

INFO     [__main__:2] http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/ucca/wiki.mrp/470004.png


In [49]:
mrp_json['input']

'In the final minute of the game, Johnson had the ball stolen by Celtics center Robert Parish, and then missed two free throws that could have won the game.'

In [50]:
mrp_parser_states

[(0,
  [(0, None),
   (1,
    (1,
     0,
     {'id': 0,
      'anchors': [{'from': 0, 'to': 2}],
      'label': 'In',
      'propagate_label': 'R'},
     [[]]))],
  [],
  [],
  [],
  [(0, 0, [(0, 0, None)])],
  [(0, True, 'R', [(0, False, 'In', 'In')])]),
 (1,
  [(0, None),
   (1,
    (1,
     0,
     {'id': 1,
      'anchors': [{'from': 3, 'to': 6}],
      'label': 'the',
      'propagate_label': 'E'},
     [[]]))],
  [],
  [],
  [],
  [(0, 0, [(0, 0, None)]), (1, 1, [(1, 1, None)])],
  [(0, True, 'R', [(0, False, 'In', 'In')]),
   (1, True, 'E', [(1, False, 'the', 'the')])]),
 (2,
  [(0, None),
   (1,
    (1,
     0,
     {'id': 2,
      'anchors': [{'from': 7, 'to': 12}],
      'label': 'final',
      'propagate_label': 'E'},
     [[]]))],
  [],
  [],
  [],
  [(0, 0, [(0, 0, None)]), (1, 1, [(1, 1, None)]), (2, 2, [(2, 2, None)])],
  [(0, True, 'R', [(0, False, 'In', 'In')]),
   (1, True, 'E', [(1, False, 'the', 'the')]),
   (2, True, 'E', [(2, False, 'final', 'final')])]),
 (3,
  

In [51]:
[(node['id'], node.get('label')) for node in mrp_json['nodes']]

[(0, 'In'),
 (1, 'the'),
 (2, 'final'),
 (3, 'minute'),
 (4, 'of'),
 (5, 'the'),
 (6, 'game'),
 (7, ','),
 (8, 'Johnson'),
 (9, 'had'),
 (10, 'the'),
 (11, 'ball'),
 (12, 'stolen'),
 (13, 'by'),
 (14, 'Celtics'),
 (15, 'center'),
 (16, 'RobertParish'),
 (17, ','),
 (18, 'and'),
 (19, 'then'),
 (20, 'missed'),
 (21, 'two'),
 (22, 'free'),
 (23, 'throws'),
 (24, 'that'),
 (25, 'could'),
 (26, 'have'),
 (27, 'won'),
 (28, 'the'),
 (29, 'game'),
 (30, '.'),
 (31, None),
 (32, None),
 (33, None),
 (34, None),
 (35, None),
 (36, None),
 (37, None),
 (38, None),
 (39, None),
 (40, None),
 (41, None)]

In [52]:
doc

'In the final minute of the game, Johnson had the ball stolen by Celtics center Robert Parish, and then missed two free throws that could have won the game.'

In [53]:
parse_json['nodes']

[{'id': 0,
  'label': 'In',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['in', 'ADP', 'IN']},
 {'id': 1,
  'label': 'the',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['the', 'DET', 'DT']},
 {'id': 2,
  'label': 'final',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['final', 'ADJ', 'JJ']},
 {'id': 3,
  'label': 'minute',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['minute', 'NOUN', 'NN']},
 {'id': 4,
  'label': 'of',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['of', 'ADP', 'IN']},
 {'id': 5,
  'label': 'the',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['the', 'DET', 'DT']},
 {'id': 6,
  'label': 'game',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['game', 'NOUN', 'NN']},
 {'id': 7,
  'label': ',',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': [',', 'PUNCT', ',']},
 {'id': 8,
  'label': 'Johnson',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['Johnson', 'PROPN', 'NNP']},
 {'id': 9,
  'label

In [54]:
[(node['id'], node['label']) for node in parse_json['nodes']]

[(0, 'In'),
 (1, 'the'),
 (2, 'final'),
 (3, 'minute'),
 (4, 'of'),
 (5, 'the'),
 (6, 'game'),
 (7, ','),
 (8, 'Johnson'),
 (9, 'had'),
 (10, 'the'),
 (11, 'ball'),
 (12, 'stolen'),
 (13, 'by'),
 (14, 'Celtics'),
 (15, 'center'),
 (16, 'Robert'),
 (17, 'Parish'),
 (18, ','),
 (19, 'and'),
 (20, 'then'),
 (21, 'missed'),
 (22, 'two'),
 (23, 'free'),
 (24, 'throws'),
 (25, 'that'),
 (26, 'could'),
 (27, 'have'),
 (28, 'won'),
 (29, 'the'),
 (30, 'game'),
 (31, '.')]

In [55]:
anchors

[(0, 2),
 (3, 6),
 (7, 12),
 (13, 19),
 (20, 22),
 (23, 26),
 (27, 31),
 (31, 32),
 (33, 40),
 (41, 44),
 (45, 48),
 (49, 53),
 (54, 60),
 (61, 63),
 (64, 71),
 (72, 78),
 (79, 85),
 (86, 92),
 (92, 93),
 (94, 97),
 (98, 102),
 (103, 109),
 (110, 113),
 (114, 118),
 (119, 125),
 (126, 130),
 (131, 136),
 (137, 141),
 (142, 145),
 (146, 149),
 (150, 154),
 (154, 155)]

### Create training instance

In [388]:
# # Only ucca
# framework = 'ucca'
# ignore_framework_set = {'amr', 'dm', 'psd', 'eds'}
# dataset = 'wiki'
# ignore_dataset_set = {}

# # Only dm
# framework = 'dm'
# ignore_framework_set = {'amr', 'ucca', 'psd', 'eds'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# # Only psd
# framework = 'psd'
# ignore_framework_set = {'amr', 'ucca', 'dm', 'eds'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# Only eds
# framework = 'eds'
# ignore_framework_set = {'amr', 'ucca', 'dm', 'psd'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# # Only psd
# framework = 'psd'
# ignore_framework_set = {'amr', 'ucca', 'dm', 'eds'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# Only AMR
framework = 'amr'
ignore_framework_set = {'psd', 'ucca', 'dm', 'eds'}
dataset = 'wsj'
ignore_dataset_set = {}

# # ucca-dm
# framework = 'ucca'
# ignore_framework_set = {'amr', 'psd', 'eds'}
# dataset = 'wiki'
# ignore_dataset_set = {}

In [57]:
# frameworks = ['amr', 'ucca', 'psd', 'dm', 'eds']
frameworks = ['amr']

In [58]:
framework_names = '-'.join([
    framework 
    for framework in frameworks 
    if framework not in ignore_framework_set
])
framework_names

'amr'

In [59]:
allennlp_tests_fixtures_output_file = os.path.join(
    args.project_root, args.mrp_test_dir, args.tests_fixtures_file_template.format(framework_names))

allennlp_framework_train_output_file = os.path.join(
    args.project_root, args.allennlp_mrp_json_file_template.format(framework_names, 'train'))

allennlp_framework_test_output_file = os.path.join(
    args.project_root, args.allennlp_mrp_json_file_template.format(framework_names, 'test'))

In [60]:
# Create tests fixture jsonl
fixture_combinations = [
#     ('ucca', 'wiki', 70),
    ('dm', 'wsj', 3)
] * 5

with open(allennlp_tests_fixtures_output_file, 'w') as wf:
    for framework, dataset, idx in fixture_combinations:
        mrp_json = framework2dataset2mrp_jsons[framework][dataset][idx]
        cid = mrp_json.get('id')
        doc = mrp_json.get('input')
        
        alignment = {}
        if framework == 'amr':
            alignment = cid2alignment[cid]  
        parse_json = dataset2cid2parse_json.get(dataset, {}).get(cid, {})

        if parse_json:
            mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
                mrp_json, 
                tokenized_parse_nodes=parse_json['nodes'],
                alignment=alignment,
            )
            companion_parser_states, companion_meta_data = mrp_json2parser_states(
                parse_json, 
                mrp_doc=doc,
                tokenized_parse_nodes=parse_json['nodes'],
            )

            data_instance = {
                'mrp_json': mrp_json,
                'parse_json': parse_json,
                'mrp_parser_states': mrp_parser_states,
                'mrp_meta_data': mrp_meta_data,
                'companion_parser_states': companion_parser_states,
                'companion_meta_data': companion_meta_data,
            }
            json_encoded_instance = json.dumps(data_instance)
            wf.write(json_encoded_instance + '\n')

In [61]:
for idx in range(20):
    mrp_json = framework2dataset2mrp_jsons[framework][dataset][idx]
    cid = mrp_json.get('id')
    if cid in dataset2cid2parse[dataset]:
        print(idx)

3
8
13
18


In [62]:
[state[-1] for state in mrp_parser_states]

[[(0, True, 'the', [(0, False, 'the', 'The')])],
 [(0, True, 'the', [(0, False, 'the', 'The')]),
  (1, True, 'asbestos', [(1, False, 'asbestos', 'asbestos')])],
 [(0, True, 'the', [(0, False, 'the', 'The')]),
  (1, True, 'asbestos', [(1, False, 'asbestos', 'asbestos')]),
  (2, False, 'fiber', 'fiber')],
 [(0, True, 'the', [(0, False, 'the', 'The')]),
  (1, True, 'asbestos', [(1, False, 'asbestos', 'asbestos')]),
  (2, False, 'fiber', 'fiber'),
  (4, True, 'crocidolite', [(4, False, 'crocidolite', 'crocidolite')])],
 [(2,
   True,
   'fiber',
   [(0, True, 'the', [(0, False, 'the', 'The')]),
    (1, True, 'asbestos', [(1, False, 'asbestos', 'asbestos')]),
    (2, False, 'fiber', 'fiber'),
    (4, True, 'crocidolite', [(4, False, 'crocidolite', 'crocidolite')])])],
 [(2,
   True,
   'fiber',
   [(0, True, 'the', [(0, False, 'the', 'The')]),
    (1, True, 'asbestos', [(1, False, 'asbestos', 'asbestos')]),
    (2, False, 'fiber', 'fiber'),
    (4, True, 'crocidolite', [(4, False, 'crocidol

In [63]:
mrp_meta_data[-1]

[(0, None),
 (1,
  (1,
   0,
   {'id': 0,
    'label': 'the',
    'properties': ['pos', 'frame'],
    'values': ['DT', 'q:i-h-h'],
    'anchors': [{'from': 0, 'to': 3}]},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 1,
    'label': 'asbestos',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 4, 'to': 12}]},
   [[]])),
 (0, None),
 (2, None),
 (0, None),
 (1,
  (1,
   0,
   {'id': 4,
    'label': 'crocidolite',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 20, 'to': 31}]},
   [[]])),
 (1,
  (4,
   2,
   {'id': 2,
    'label': 'fiber',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 13, 'to': 18}]},
   [[{'source': 0,
      'target': 2,
      'label': 'BV',
      'id': 2,
      'parent': 2,
      'child': 0}],
    [{'source': 1,
      'target': 2,
      'label': 'compound',
      'id': 1,
      'parent': 2,
      'child': 1}],
    [],
    [{'source': 4,
    

In [64]:
doc

'The asbestos fiber, crocidolite, is unusually resilient once it enters the lungs, with even brief exposures to it causing symptoms that show up decades later, researchers said.'

In [65]:
parse_json

{'id': '20003002',
 'tops': [30],
 'nodes': [{'id': 0,
   'label': 'The',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['the', 'DET', 'DT']},
  {'id': 1,
   'label': 'asbestos',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['asbestos', 'NOUN', 'NN']},
  {'id': 2,
   'label': 'fiber',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['fiber', 'NOUN', 'NN']},
  {'id': 3,
   'label': ',',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': [',', 'PUNCT', ',']},
  {'id': 4,
   'label': 'crocidolite',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['crocidolite', 'NOUN', 'NN']},
  {'id': 5,
   'label': ',',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': [',', 'PUNCT', ',']},
  {'id': 6,
   'label': 'is',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['be', 'VERB', 'VBZ']},
  {'id': 7,
   'label': 'unusually',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['unusually', 'ADV', 'RB']},
  {'id': 8,
   'label': 'resil

In [66]:
[n['values'][2] for n in parse_json['nodes']]

['DT',
 'NN',
 'NN',
 ',',
 'NN',
 ',',
 'VBZ',
 'RB',
 'JJ',
 'IN',
 'PRP',
 'VBZ',
 'DT',
 'NNS',
 ',',
 'IN',
 'RB',
 'JJ',
 'NNS',
 'TO',
 'PRP',
 'VBG',
 'NNS',
 'WDT',
 'VBP',
 'RP',
 'NNS',
 'RB',
 ',',
 'NNS',
 'VBD',
 '.']

In [68]:
# Create train jsonl
if os.path.isfile(allennlp_framework_train_output_file) and os.path.isfile(
    allennlp_framework_train_output_file):
    logger.info('allennlp_train_output_file found, stop generation')
else:
#     pass
# if 1==1:
    data_size = 0
    with open(allennlp_framework_train_output_file, 'w') as train_wf:
        with open(allennlp_framework_test_output_file, 'w') as test_wf:
            for _, dataset, idx, mrp_json in tqdm(mrp_dataset.mrp_json_generator(
                ignore_framework_set=ignore_framework_set,
                ignore_dataset_set=ignore_dataset_set,
                data_size_limit=args.data_size_limit * 2
            )):
                cid = mrp_json.get('id')
                doc = mrp_json.get('input')

                framework = mrp_json.get('framework')
                alignment = {}
                if framework == 'amr':
                    alignment = cid2alignment[cid]  
                parse_json = dataset2cid2parse_json.get(dataset, {}).get(cid, {})

                if parse_json:
                    mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
                        mrp_json, 
                        tokenized_parse_nodes=parse_json['nodes'],
                        alignment=alignment,
                    )
                    companion_parser_states, companion_meta_data = mrp_json2parser_states(
                        parse_json, 
                        mrp_doc=doc,
                        tokenized_parse_nodes=parse_json['nodes'],
                    )

                    # Continue if error
                    if not mrp_parser_states:
                        continue

                    data_instance = {
                        'mrp_json': mrp_json,
                        'parse_json': parse_json,
                        'mrp_parser_states': mrp_parser_states,
                        'mrp_meta_data': mrp_meta_data,
                        'companion_parser_states': companion_parser_states,
                        'companion_meta_data': companion_meta_data,
                    }
                    json_encoded_instance = json.dumps(data_instance)
                    if idx <= args.data_size_limit:
                        train_wf.write(json_encoded_instance + '\n')
                    else:
                        test_wf.write(json_encoded_instance + '\n')

INFO     [__main__:4] allennlp_train_output_file found, stop generation


### Generate test mrp json data

In [69]:
allennlp_evaluation_output_file = os.path.join(
    args.project_root, args.mrp_data_dir, args.evaluation_allennlp_mrp_json_file)
allennlp_evaluation_output_file

'/data/proj29_ds1/home/slai/mrp2019/data/evaluation/allennlp-evaluation.mrp'

In [70]:
if os.path.isfile(allennlp_evaluation_output_file):
    logger.info('allennlp_evaluation_output_file found, stop generation')
else:
    with open(allennlp_evaluation_output_file, 'w') as wf:
        for mrp_json in tqdm(test_mrp_jsons):
            cid = mrp_json.get('id')
            doc = mrp_json.get('input')
            frameworks = mrp_json.get('targets')
            parse_json = test_parse_jsons[cid]

            for framework in frameworks:
                companion_parser_states, companion_meta_data = mrp_json2parser_states(
                    parse_json, 
                    mrp_doc=doc,
                    tokenized_parse_nodes=parse_json['nodes'],
                )

                data_instance = {
                    'cid': cid,
                    'doc': doc,
                    'framework': framework,
                    'mrp_json': mrp_json,
                    'parse_json': parse_json,
                    'companion_parser_states': companion_parser_states,
                    'companion_meta_data': companion_meta_data,
                }
                json_encoded_instance = json.dumps(data_instance)
                wf.write(json_encoded_instance + '\n')

INFO     [__main__:2] allennlp_evaluation_output_file found, stop generation


### Test allennlp dataset reader

In [402]:
import torch.optim as optim

from allennlp.common.file_utils import cached_path
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders.embedding import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules.feedforward import FeedForward

from allennlp.training.metrics import CategoricalAccuracy

from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

import json
import logging
from typing import Dict

from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import LabelField, TextField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer
from allennlp.models import Model
from overrides import overrides

In [403]:
from mrp_library.dataset_readers.mrp_jsons_actions import MRPDatasetActionReader

In [404]:
reader = MRPDatasetActionReader()

In [405]:
allennlp_cache_dataset_dir = os.path.join(
    args.project_root, args.mrp_data_dir, args.allennlp_cache_dir)
allennlp_cache_dataset_dir

'/data/proj29_ds1/home/slai/mrp2019/data/allennlp-cache'

In [406]:
# reader.cache_data(allennlp_cache_dataset_dir)

In [407]:
train_dataset = reader.read(cached_path(allennlp_framework_train_output_file))




















0it [00:00, ?it/s]INFO     [mrp_library.dataset_readers.mrp_jsons_actions:139] Reading instances from lines in file at: /data/proj29_ds1/home/slai/mrp2019/allennlp-mrp-json-big-amr-train.jsonl
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 1)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 2)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 3)



















115it [00:00, 1149.14it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4)



















470it [00:00, 1419.24it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 6)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 7)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 8)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:43

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 85)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 86)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 87)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 88)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 89)



















3015it [00:01, 2149.17it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 90)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 91)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 92)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 93)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 94)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 95)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 172)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 173)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 174)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 175)



















6151it [00:02, 2020.26it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 176)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 177)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 178)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 179)



















6396it [00:02, 2130.27it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 180)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 181)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 182)
WARNIN

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 256)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 257)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 258)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 259)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 260)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 261)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 262)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 263)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 264)



















9579it [00:04, 2370.73it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 265)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 266)
WARNING  [mrp_library.dataset_readers.mrp_jsons_acti

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 340)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 341)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 342)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 343)



















12910it [00:38, 13.01it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 344)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 345)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 346)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 347)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 348)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 349)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 350)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actio

...

244293it [03:00, 2993.29it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4284)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4285)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4286)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4287)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4288)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4289)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4290)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4291)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4292)



















244595it [03:00, 2882.57it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4293)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4294)
WARNING  [

249041it [03:01, 3196.24it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4366)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4367)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4368)



















249443it [03:01, 3402.62it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4369)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4370)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4371)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4372)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4373)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4374)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4375)



















249820it [03:01, 3504.88it/s]WARNING  [mrp_library.dataset_readers.mrp_jso




















253499it [03:02, 3462.52it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4450)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4451)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4452)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4453)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4454)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4455)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4456)



















253858it [03:02, 3499.22it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4457)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4458)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4459)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4534)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4535)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4536)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4537)



















258077it [03:03, 3545.03it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4538)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4539)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4540)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4541)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4542)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4543)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4544)
WARNING  [mrp_library.dataset_readers.m

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4619)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4620)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4621)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4622)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4623)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4624)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4625)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4626)



















262039it [03:05, 3227.81it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4627)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4628)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4629)
WARNING  [mrp_library.dataset_readers.m

In [408]:
# test_dataset = reader.read(cached_path(allennlp_train_output_file))
test_dataset = reader.read(cached_path(allennlp_framework_test_output_file))




















0it [00:00, ?it/s]INFO     [mrp_library.dataset_readers.mrp_jsons_actions:139] Reading instances from lines in file at: /data/proj29_ds1/home/slai/mrp2019/allennlp-mrp-json-big-amr-test.jsonl
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4700)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4701)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4702)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4703)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4704)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4705)



















462it [00:00, 4551.03it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4706)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4707)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 470

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4784)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4785)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4786)



















4198it [00:01, 3686.23it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4787)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4788)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4789)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4790)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4791)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4792)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4793)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4794)
WARNING  [mrp_library.dataset_readers.mrp

8157it [00:02, 2998.28it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4869)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4870)



















8517it [00:02, 3155.96it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4871)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4872)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4873)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4874)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4875)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4876)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4877)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4878)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4879)
WARNING  [mrp_




















12777it [00:03, 3125.85it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4952)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4953)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4954)



















13157it [00:03, 3301.42it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4955)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4956)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4957)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4958)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4959)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4960)



















13515it [00:03, 3379.14it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 4961)
WARNING  [mrp_library.dataset

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5037)



















16835it [00:04, 2812.78it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5038)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5039)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5040)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5041)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5042)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5043)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5044)



















17145it [00:05, 2892.82it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5045)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5046)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count',

22011it [00:06, 3777.46it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5117)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5118)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5119)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5120)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5121)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5122)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5123)



















22391it [00:06, 3654.71it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5124)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5125)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5126)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5127)
WARNING  [mr

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5199)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5200)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5201)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5202)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5203)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5204)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5205)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5206)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5207)



















27232it [00:07, 3554.85it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5208)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5209)
WARNING  [mrp_library.dataset_readers.mr




















30854it [00:08, 3458.06it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5286)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5287)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5288)



















31210it [00:09, 3478.75it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5289)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5290)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5291)



















31634it [00:09, 3666.94it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5292)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5293)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5294)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5295)
WARNING  [mrp_library.dataset

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5369)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5370)



















35817it [00:10, 3406.21it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5371)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5372)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5373)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5374)



















36195it [00:10, 3480.84it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5375)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5376)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5377)



















36589it [00:10, 3606.43it/s]WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:431] ('error count', 5378)
WARNING  [mrp_library.dataset

In [409]:
tests_fixtures_dataset = reader.read(cached_path(allennlp_tests_fixtures_output_file))




















0it [00:00, ?it/s]INFO     [mrp_library.dataset_readers.mrp_jsons_actions:139] Reading instances from lines in file at: /data/proj29_ds1/home/slai/mrp2019/src/tests/fixtures/amr-test.jsonl



















382it [00:00, 3816.79it/s]


















385it [00:00, 3668.13it/s]

In [410]:
non_padded_namespaces = [
    'action_type_labels',
]
label_fields = [
    'resolved_label_root_label',
    'resolved_label_edge_labels',
]
for framework in frameworks:
    for label_field in label_fields:
        non_padded_namespaces.append('{}_{}'.format(framework, label_field))

In [411]:
non_padded_namespaces

['action_type_labels',
 'amr_resolved_label_root_label',
 'amr_resolved_label_edge_labels',
 'ucca_resolved_label_root_label',
 'ucca_resolved_label_edge_labels',
 'psd_resolved_label_root_label',
 'psd_resolved_label_edge_labels',
 'dm_resolved_label_root_label',
 'dm_resolved_label_edge_labels',
 'eds_resolved_label_root_label',
 'eds_resolved_label_edge_labels']

In [416]:
vocab = Vocabulary.from_instances(
    train_dataset + test_dataset + tests_fixtures_dataset,
    non_padded_namespaces=non_padded_namespaces,
)

INFO     [allennlp.data.vocabulary:396] Fitting token dictionary from dataset.



















  0%|          | 0/304278 [00:00<?, ?it/s]


















  0%|          | 729/304278 [00:00<00:41, 7289.33it/s]


















  0%|          | 1168/304278 [00:00<00:49, 6083.44it/s]


















  1%|          | 1808/304278 [00:00<00:49, 6172.03it/s]


















  1%|          | 2319/304278 [00:00<00:52, 5806.46it/s]


















  1%|          | 2816/304278 [00:00<00:54, 5526.63it/s]


















  1%|          | 3399/304278 [00:00<00:53, 5613.05it/s]


















  1%|▏         | 4061/304278 [00:00<00:51, 5822.55it/s]


















  2%|▏         | 4595/304278 [00:00<00:53, 5580.40it/s]


















  2%|▏         | 5255/304278 [00:00<00:51, 5848.29it/s]


















  2%|▏         | 5903/304278 [00:01<00:49, 6020.56it/s]


















  2%|▏         | 6493/304278 [00:01<00:58, 5058.32it/s]


















  2%|▏         | 7162/304278

 15%|█▍        | 45473/304278 [00:06<00:30, 8560.17it/s]


















 15%|█▌        | 46336/304278 [00:06<00:30, 8572.55it/s]


















 16%|█▌        | 47307/304278 [00:06<00:28, 8884.21it/s]


















 16%|█▌        | 48202/304278 [00:06<00:28, 8875.69it/s]


















 16%|█▌        | 49113/304278 [00:06<00:28, 8942.53it/s]


















 16%|█▋        | 50124/304278 [00:06<00:27, 9263.20it/s]


















 17%|█▋        | 51056/304278 [00:07<00:27, 9239.11it/s]


















 17%|█▋        | 51984/304278 [00:07<00:28, 8904.95it/s]


















 17%|█▋        | 52907/304278 [00:07<00:27, 8997.83it/s]


















 18%|█▊        | 53940/304278 [00:07<00:26, 9359.18it/s]


















 18%|█▊        | 55004/304278 [00:07<00:25, 9709.07it/s]


















 18%|█▊        | 55983/304278 [00:07<00:25, 9557.54it/s]


















 19%|█▊        | 56945/304278 [00:07<00:26, 9252.70it/s]


















 19%|█▉        | 57877/30

 34%|███▍      | 103843/304278 [00:12<00:22, 8962.91it/s]


















 34%|███▍      | 104866/304278 [00:12<00:21, 9308.65it/s]


















 35%|███▍      | 105812/304278 [00:13<00:21, 9353.29it/s]


















 35%|███▌      | 106752/304278 [00:13<00:21, 9287.66it/s]


















 35%|███▌      | 107694/304278 [00:13<00:21, 9326.05it/s]


















 36%|███▌      | 108838/304278 [00:13<00:19, 9870.35it/s]


















 36%|███▌      | 109920/304278 [00:13<00:19, 10137.13it/s]


















 36%|███▋      | 110943/304278 [00:13<00:19, 9713.78it/s] 


















 37%|███▋      | 112098/304278 [00:13<00:18, 10199.41it/s]


















 37%|███▋      | 113132/304278 [00:13<00:19, 9692.49it/s] 


















 38%|███▊      | 114116/304278 [00:13<00:19, 9540.54it/s]


















 38%|███▊      | 115099/304278 [00:14<00:19, 9622.38it/s]


















 38%|███▊      | 116069/304278 [00:14<00:19, 9592.83it/s]


















 38%|███

 53%|█████▎    | 162142/304278 [00:19<00:17, 8327.45it/s]


















 54%|█████▎    | 163001/304278 [00:19<00:16, 8403.82it/s]


















 54%|█████▍    | 164115/304278 [00:19<00:15, 9072.03it/s]


















 54%|█████▍    | 165172/304278 [00:19<00:14, 9473.08it/s]


















 55%|█████▍    | 166161/304278 [00:19<00:14, 9594.03it/s]


















 55%|█████▍    | 167133/304278 [00:19<00:14, 9351.83it/s]


















 55%|█████▌    | 168106/304278 [00:19<00:14, 9461.52it/s]


















 56%|█████▌    | 169060/304278 [00:20<00:14, 9124.65it/s]


















 56%|█████▌    | 169980/304278 [00:20<00:14, 9095.89it/s]


















 56%|█████▌    | 170895/304278 [00:20<00:15, 8859.36it/s]


















 56%|█████▋    | 171786/304278 [00:20<00:15, 8664.18it/s]


















 57%|█████▋    | 172666/304278 [00:20<00:15, 8704.21it/s]


















 57%|█████▋    | 173540/304278 [00:20<00:15, 8566.88it/s]


















 57%|█████▋ 

 70%|███████   | 213374/304278 [00:25<00:11, 8090.70it/s]


















 70%|███████   | 214240/304278 [00:25<00:10, 8252.03it/s]


















 71%|███████   | 215073/304278 [00:25<00:10, 8267.74it/s]


















 71%|███████   | 215918/304278 [00:25<00:10, 8319.85it/s]


















 71%|███████   | 216754/304278 [00:26<00:10, 8052.48it/s]


















 72%|███████▏  | 217564/304278 [00:26<00:10, 7920.99it/s]


















 72%|███████▏  | 218427/304278 [00:26<00:10, 8120.28it/s]


















 72%|███████▏  | 219243/304278 [00:26<00:10, 7964.94it/s]


















 72%|███████▏  | 220043/304278 [00:26<00:10, 7802.82it/s]


















 73%|███████▎  | 220924/304278 [00:26<00:10, 8077.18it/s]


















 73%|███████▎  | 221746/304278 [00:26<00:10, 8118.23it/s]


















 73%|███████▎  | 222608/304278 [00:26<00:09, 8262.12it/s]


















 73%|███████▎  | 223438/304278 [00:26<00:09, 8242.94it/s]


















 74%|███████

 89%|████████▊ | 270033/304278 [00:32<00:03, 10314.87it/s]


















 89%|████████▉ | 271128/304278 [00:32<00:03, 10496.39it/s]


















 89%|████████▉ | 272181/304278 [00:32<00:03, 10437.99it/s]


















 90%|████████▉ | 273227/304278 [00:32<00:03, 9826.73it/s] 


















 90%|█████████ | 274242/304278 [00:32<00:03, 9919.59it/s]


















 90%|█████████ | 275247/304278 [00:32<00:02, 9955.36it/s]


















 91%|█████████ | 276247/304278 [00:32<00:02, 9492.71it/s]


















 91%|█████████ | 277204/304278 [00:32<00:02, 9254.24it/s]


















 91%|█████████▏| 278137/304278 [00:32<00:02, 8976.05it/s]


















 92%|█████████▏| 279271/304278 [00:32<00:02, 9572.87it/s]


















 92%|█████████▏| 280351/304278 [00:33<00:02, 9835.04it/s]


















 92%|█████████▏| 281348/304278 [00:33<00:02, 9279.33it/s]


















 93%|█████████▎| 282292/304278 [00:33<00:02, 8409.92it/s]


















 93%|███

In [417]:
vocab.print_statistics()

INFO     [allennlp.data.vocabulary:664] Printed vocabulary statistics are only for the part of the vocabulary generated from instances. If vocabulary is constructed by extending saved vocabulary with dataset instances, the directly loaded portion won't be considered here.




----Vocabulary Statistics----


Top 10 most frequent tokens in namespace 'word':
	Token: the		Frequency: 1251893
	Token: ,		Frequency: 748168
	Token: .		Frequency: 627722
	Token: of		Frequency: 594538
	Token: and		Frequency: 488754
	Token: a		Frequency: 434956
	Token: in		Frequency: 431238
	Token: to		Frequency: 424656
	Token: be		Frequency: 368688
	Token: that		Frequency: 283152

Top 10 longest tokens in namespace 'word':
	Token: href=”http://democratherald.com/news/opinion/editorial/editorial-once-again-fluoridation/article_e82f150c-6263-11e1-aab4-001871e3ce6c.html#ixzz1o0ksmtvn”>editorial		length: 163	Frequency: 64
	Token: href=”http://mochamanual.com/mochamanual/trying-to-conceive/are-your-fortified-the-facts-about-folic-acid-what-every-black-woman-needs-to-know.html”>are		length: 152	Frequency: 16
	Token: href=”http://content.usatoday.com/communities/entertainment/post/2012/02/chris-christie-criticized-for-whitney-houston-flag-plan/1#.tz1agdr5msm”>chris		length: 150	Frequency: 8

In [418]:
assert vocab.get_vocab_size('action_type_labels') == 3

In [419]:
namespaces = [
    'token_node_label',
    'ucca_resolved_label_edge_labels',
    'ucca_resolved_label_root_label',
    'dm_resolved_label_edge_labels',
    'dm_resolved_label_root_label',
    'word',
    'pos',
    'label',
]

In [420]:
for namespace in namespaces:
    logger.info((namespace, vocab.get_vocab_size(namespace)))

INFO     [__main__:2] ('token_node_label', 19043)
INFO     [__main__:2] ('ucca_resolved_label_edge_labels', 0)
INFO     [__main__:2] ('ucca_resolved_label_root_label', 0)
INFO     [__main__:2] ('dm_resolved_label_edge_labels', 6)
INFO     [__main__:2] ('dm_resolved_label_root_label', 21)
INFO     [__main__:2] ('word', 23348)
INFO     [__main__:2] ('pos', 70)
INFO     [__main__:2] ('label', 2)


### Test model

In [421]:
from mrp_library.models.resolver import Resolver
from mrp_library.iterators.same_instance_type_framework_stack_len_iterator import SameInstanceTypeFrameworkStackLenIterator

from allennlp.nn import InitializerApplicator, RegularizerApplicator, util
from allennlp.nn.activations import Activation
from allennlp.common.params import Params
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.modules.seq2vec_encoders.pytorch_seq2vec_wrapper import PytorchSeq2VecWrapper
from allennlp.modules.seq2seq_encoders.pytorch_seq2seq_wrapper import PytorchSeq2SeqWrapper

In [422]:
if torch.cuda.is_available() and False:
    cuda_device = args.cuda_device
else:
    cuda_device = -1
cuda_device

-1

In [423]:
def _cuda(variable, cuda_device):
    if cuda_device != -1:
        variable = variable.cuda(cuda_device)
    return variable

### Define parameters

In [675]:
bidirectional = True
EMBEDDING_DIM = 100
HIDDEN_DIM = 50

In [676]:
# frameworks = ['dm', 'psd', 'eds', 'ucca', 'amr']
frameworks = ['amr', 'ucca', 'psd', 'dm', 'eds']

# field_types = ['word', 'pos', 
#                'ucca_word', 'ucca_pos',
#                'dm_word', 'dm_pos', 
#                'resolved', 'token_node_label', 'token_node_prev_action']

field_type2embedder = {}
field_type2seq2vec_encoder = {}
field_type2seq2seq_encoder = {}

In [677]:
glove_embedding_group2field_types = {
    'word': ['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word'],
    'token_node_label': ['token_node_label'],
}
for field in ['word']:
    for framework in frameworks:
        glove_embedding_group2field_types[field].append('resolve_{}_{}'.format(framework, field))

embedding_group2field_types = {
    'pos': ['pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos'],
    'resolved': ['resolved', 'token_node_resolved'],
    'token_node_prev_action': ['action', 'token_node_prev_action'],
    'template': ['template'],
}
for field in ['pos']:
    for framework in frameworks:
        embedding_group2field_types[field].append('resolve_{}_{}'.format(framework, field))

seq2vec_group2field_types = {
    'parse_word': ['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word'],
    'parse_pos': ['pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos'],
    'resolved': ['resolved', 'token_node_resolved'],
    'token_node_label': ['token_node_label'],
    'action': ['action', 'token_node_prev_action'],
}
for field in ['word', 'pos']:
    for framework in frameworks:
        full_name = 'resolve_{}_{}'.format(framework, field)
        seq2vec_group2field_types[full_name] = [full_name]

seq2seq_group2field_types = {
    'parse_word': ['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word'],
    'parse_pos': ['pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos'],
    'resolved': ['resolved', 'token_node_resolved'],
    'token_node_label': ['token_node_label'],
    'action': ['action', 'token_node_prev_action'],
}
for field in ['word', 'pos']:
    for framework in frameworks:
        full_name = 'resolve_{}_{}'.format(framework, field)
        seq2seq_group2field_types[full_name] = [full_name]

In [678]:
embedding_group2field_types.keys()

dict_keys(['pos', 'resolved', 'token_node_prev_action', 'template'])

In [679]:
# Embedding.from_params(
#     vocab=vocab,
#     params = Params({
#         'vocab_namespace': embedding_group,
#         'pretrained_file': '../data/glove.6B.100d.txt',
#         'embedding_dim' : EMBEDDING_DIM,
#     })
# )

In [680]:
for embedding_group, field_types in glove_embedding_group2field_types.items():
    embedding = _cuda(
        Embedding.from_params(
            vocab=vocab,
            params = Params({
                'vocab_namespace': embedding_group,
                'pretrained_file': '../data/glove.6B.100d.txt',
                'embedding_dim' : EMBEDDING_DIM,
                "trainable": True,
            })
        ), cuda_device)
    logger.info((embedding_group, vocab.get_vocab_size(embedding_group)))
    embedder = BasicTextFieldEmbedder({embedding_group: embedding})
    for field_type in field_types:
        field_type2embedder[field_type] = embedder

for embedding_group, field_types in embedding_group2field_types.items():
    embedding = _cuda(
        Embedding(
            num_embeddings=vocab.get_vocab_size(embedding_group),
            embedding_dim=EMBEDDING_DIM
        ), cuda_device)
    logger.info((embedding_group, vocab.get_vocab_size(embedding_group)))
    embedder = BasicTextFieldEmbedder({embedding_group: embedding})
    for field_type in field_types:
        field_type2embedder[field_type] = embedder
    
for seq2vec_group, field_types in seq2vec_group2field_types.items():
    seq2vec = PytorchSeq2VecWrapper(
        _cuda(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True, bidirectional=bidirectional), cuda_device)
    )
    for field_type in field_types:
        field_type2seq2vec_encoder[field_type] = seq2vec

for seq2seq_group, field_types in seq2seq_group2field_types.items():
    seq2seq = PytorchSeq2SeqWrapper(
        _cuda(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True, bidirectional=bidirectional), cuda_device)
    )
    for field_type in field_types:
        field_type2seq2seq_encoder[field_type] = seq2seq

INFO     [allennlp.common.params:252] num_embeddings = None
INFO     [allennlp.common.params:252] vocab_namespace = word
INFO     [allennlp.common.params:252] embedding_dim = 100
INFO     [allennlp.common.params:252] pretrained_file = ../data/glove.6B.100d.txt
INFO     [allennlp.common.params:252] projection_dim = None
INFO     [allennlp.common.params:252] trainable = True
INFO     [allennlp.common.params:252] padding_index = None
INFO     [allennlp.common.params:252] max_norm = None
INFO     [allennlp.common.params:252] norm_type = 2.0
INFO     [allennlp.common.params:252] scale_grad_by_freq = False
INFO     [allennlp.common.params:252] sparse = False
INFO     [allennlp.modules.token_embedders.embedding:393] Reading pretrained embeddings from file






















0it [00:00, ?it/s]





















2626it [00:00, 26259.84it/s]





















5922it [00:00, 27965.17it/s]





















10193it [00:00, 31194.37it/s]





















17253it [00:00, 37467.08it/s

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token ▪ was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Thailand was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Defense was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Africa was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Council was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Israel was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token <NO-MORE-PARSE-NODE> was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Indone

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Office was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Committee was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Health was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token AFP was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Energy was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Amnesty was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Katrina was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Hussein was not f

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Beijing was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token River was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Party was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Press was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Federation was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Saudi was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Arafat was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Mohammed was not f

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Asian was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Downer was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Colombia was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token offens was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Bureau was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Arab was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Black was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token 030000 was not found 

...

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token foreclose-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Dodd/Frank was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Furious was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Record was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Settlements was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Pelting was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Obama-Derangement was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:43

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Rip was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token href=”http://www.forbes.com/sites/peterferrara/2012/01/12/the-worst-economic-recovery-since-the-great-depression/2/”>The was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Nope was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Lady was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token shaft-00 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Holder was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token swear-01 was not found in the embedding f

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token JULY was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Bachmann was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token SEPTEMBER was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token OCTOBER was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token NOVEMBER was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token perjure-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Newts was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token blowjobs w

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token neutralize-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token malignant-02 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token concomitants was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token locate-02 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token indispensable-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token mainland-bound was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token cybersphere was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embed

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token practically-04 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token denounce-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Glaciology was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token bowl-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token shuttle-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token toil-01 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token have-degree-of-resemblance-91 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embe

In [681]:
field_type2embedder['word'].state_dict()['token_embedder_word.weight'].shape

torch.Size([23348, 100])

In [682]:
# word_embedding = Embedding(num_embeddings=vocab.get_vocab_size('word'),
#                             embedding_dim=EMBEDDING_DIM)
# pos_embedding = Embedding(num_embeddings=vocab.get_vocab_size('pos'),
#                             embedding_dim=EMBEDDING_DIM)

# word_embedder = BasicTextFieldEmbedder({
#     "word": word_embedding,
#     "pos": pos_embedding,
# })
# parse_label = {
#     'word': torch.LongTensor(
#         [
#             [ 1,  0,  3,  7,  2,  9,  4],
#             [ 0,  0,  5,  0,  0,  0,  4]
#         ]
#     ),
#     'pos': torch.LongTensor(
#         [
#             [ 1,  0,  3,  7,  2,  9,  4],
#             [ 0,  0,  5,  0,  0,  0,  4]
#         ]
#     )
# }

In [683]:
# embedded_parse_label = word_embedder(parse_label)

In [684]:
# embedded_parse_label.shape

In [685]:
vocab.get_vocab_size('resolved_label_edge_labels')

2

In [686]:
vocab.get_vocab_size('dm_resolved_label_edge_labels')

6

In [687]:
vocab.get_vocab_size('dm_resolved_label_root_label')

21

In [688]:
vocab.get_vocab_size('action_type_labels')

3

In [689]:
if bidirectional:
    LSTM_DIRECTIONS = 2
else:
    LSTM_DIRECTIONS = 1

In [690]:
vocab.get_vocab_size('action_type_labels')

3

In [692]:
frameworks

['amr', 'ucca', 'psd', 'dm', 'eds']

In [693]:
action_classifier_params = Params({
  "input_dim": HIDDEN_DIM * 15 * LSTM_DIRECTIONS + 4 * EMBEDDING_DIM,
  "num_layers": 2,
  "hidden_dims": [50, vocab.get_vocab_size('action_type_labels')],
  "activations": ["sigmoid", "linear"],
  "dropout": [0.0, 0.0]
})

action_num_pop_classifier_params = Params({
  "input_dim": HIDDEN_DIM * 2 * LSTM_DIRECTIONS + EMBEDDING_DIM * 2,
  "num_layers": 2,
  "hidden_dims": [50, 1],
  "activations": ["sigmoid", "linear"],
  "dropout": [0.0, 0.0]
})

framework2field_type2feedforward_params = {}
for framework in frameworks:
    framework2field_type2feedforward_params[framework] = {
        'child_edges': Params({
            "input_dim": HIDDEN_DIM * 4 * LSTM_DIRECTIONS,
            "num_layers": 2,
            "hidden_dims": [100, vocab.get_vocab_size('{}_resolved_label_edge_labels'.format(framework))],
            "activations": ["sigmoid", "linear"],
            "dropout": [0.0, 0.0]
        }),
        'root_label': Params({
            "input_dim": EMBEDDING_DIM * 4,
            "num_layers": 2,
            "hidden_dims": [100, vocab.get_vocab_size('{}_resolved_label_root_label'.format(framework))],
            "activations": ["sigmoid", "linear"],
            "dropout": [0.0, 0.0]
        }),
    }

In [694]:
action_classifier_feedforward = FeedForward.from_params(action_classifier_params)
action_classifier_feedforward = _cuda(action_classifier_feedforward, cuda_device)

action_num_pop_classifier_feedforward = FeedForward.from_params(action_num_pop_classifier_params)
action_num_pop_classifier_feedforward = _cuda(action_num_pop_classifier_feedforward, cuda_device)

INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.modules.feedforward.FeedForward'> from params {'input_dim': 1900, 'num_layers': 2, 'hidden_dims': [50, 3], 'activations': ['sigmoid', 'linear'], 'dropout': [0.0, 0.0]} and extras set()
INFO     [allennlp.common.params:252] input_dim = 1900
INFO     [allennlp.common.params:252] num_layers = 2
INFO     [allennlp.common.params:252] hidden_dims = [50, 3]
INFO     [allennlp.common.params:252] hidden_dims = [50, 3]
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params ['sigmoid', 'linear'] and extras set()
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params sigmoid and extras set()
INFO     [allennlp.common.params:252] type = sigmoid
IN

In [695]:
framework2field_type2feedforward = {}

for framework, field_type2feedforward_params in framework2field_type2feedforward_params.items():
    framework2field_type2feedforward[framework] = {}
    for field_type, feedforward_params in field_type2feedforward_params.items():
        feedforward_classifier = FeedForward.from_params(
            feedforward_params)
        feedforward_classifier = _cuda(feedforward_classifier, cuda_device)
        framework2field_type2feedforward[framework][field_type] = feedforward_classifier
    framework2field_type2feedforward[framework] = torch.nn.ModuleDict(framework2field_type2feedforward[framework])

framework2field_type2feedforward = torch.nn.ModuleDict(framework2field_type2feedforward)

INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.modules.feedforward.FeedForward'> from params {'input_dim': 400, 'num_layers': 2, 'hidden_dims': [100, 105], 'activations': ['sigmoid', 'linear'], 'dropout': [0.0, 0.0]} and extras set()
INFO     [allennlp.common.params:252] input_dim = 400
INFO     [allennlp.common.params:252] num_layers = 2
INFO     [allennlp.common.params:252] hidden_dims = [100, 105]
INFO     [allennlp.common.params:252] hidden_dims = [100, 105]
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params ['sigmoid', 'linear'] and extras set()
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params sigmoid and extras set()
INFO     [allennlp.common.params:252] type = sig

INFO     [allennlp.common.params:252] num_layers = 2
INFO     [allennlp.common.params:252] hidden_dims = [100, 6]
INFO     [allennlp.common.params:252] hidden_dims = [100, 6]
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params ['sigmoid', 'linear'] and extras set()
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params sigmoid and extras set()
INFO     [allennlp.common.params:252] type = sigmoid
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params linear and extras set()
INFO     [allennlp.common.params:252] type = linear
INFO     [allennlp.common.params:252] dropout = [0.0, 0.0]
INFO     [allennlp.common.params:252] dropout = [0.0, 0.0]
INFO  

In [444]:
field_type2embedder.keys()

dict_keys(['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word', 'resolve_amr_word', 'resolve_ucca_word', 'resolve_psd_word', 'resolve_dm_word', 'resolve_eds_word', 'token_node_label', 'pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos', 'resolve_amr_pos', 'resolve_ucca_pos', 'resolve_psd_pos', 'resolve_dm_pos', 'resolve_eds_pos', 'resolved', 'token_node_resolved', 'action', 'token_node_prev_action', 'template'])

In [445]:
field_type2seq2vec_encoder.keys()

dict_keys(['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word', 'pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos', 'resolved', 'token_node_resolved', 'token_node_label', 'action', 'token_node_prev_action', 'resolve_amr_word', 'resolve_ucca_word', 'resolve_psd_word', 'resolve_dm_word', 'resolve_eds_word', 'resolve_amr_pos', 'resolve_ucca_pos', 'resolve_psd_pos', 'resolve_dm_pos', 'resolve_eds_pos'])

In [446]:
framework2field_type2feedforward.keys()

odict_keys(['amr', 'dm', 'eds', 'psd', 'ucca'])

In [447]:
framework2field_type2feedforward['eds'].keys()

odict_keys(['child_edges', 'root_label'])

In [448]:
field_type = 'word'

In [449]:
parse_label = {
    field_type: _cuda(
        torch.LongTensor(
            [
                [ 1,  0,  3,  7,  2,  9,  4],
                [ 0,  0,  5,  0,  0,  0,  4]
            ]
        ),
        cuda_device
    )
}
embedded_parse_label = field_type2embedder[field_type](parse_label)

In [450]:
feature_mask = util.get_text_field_mask(parse_label)

In [451]:
seq2vec_encoder = field_type2seq2vec_encoder[field_type]

In [452]:
encoded_feature = seq2vec_encoder(embedded_parse_label, feature_mask)

In [453]:
encoded_features = [encoded_feature] * 15 + [encoded_feature] * 4

In [454]:
# torch.cat(encoded_features, dim=-1)

In [455]:
actions

[(0, None),
 (1,
  (1,
   0,
   {'id': 0,
    'anchors': [{'from': 0, 'to': 2}],
    'label': 'In',
    'propagate_label': 'R'},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 1,
    'anchors': [{'from': 3, 'to': 6}],
    'label': 'the',
    'propagate_label': 'E'},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 2,
    'anchors': [{'from': 7, 'to': 12}],
    'label': 'final',
    'propagate_label': 'E'},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 3,
    'anchors': [{'from': 13, 'to': 19}],
    'label': 'minute',
    'propagate_label': 'C'},
   [[]])),
 (1,
  (4,
   3,
   {'id': 31, 'propagate_label': 'E'},
   [[{'source': 31,
      'target': 0,
      'label': 'R',
      'id': 31,
      'parent': 31,
      'child': 0}],
    [{'source': 31,
      'target': 1,
      'label': 'E',
      'id': 28,
      'parent': 31,
      'child': 1}],
    [{'source': 31,
      'target': 2,
      'label': 'E',
      'id': 21,
      'parent': 31,
      'child': 2}],
    [{'source': 31,
      't

In [456]:
logits = action_classifier_feedforward(torch.cat(encoded_features, dim=-1))

In [457]:
logits.shape

torch.Size([2, 3])

In [458]:
label = torch.tensor([1, 0])

In [459]:
# loss_func = torch.nn.CrossEntropyLoss()
# loss = loss_func(logits, label)

In [460]:
Resolver = None

In [461]:
from mrp_library.models.resolver import Resolver
from mrp_library.iterators.same_instance_type_framework_stack_len_iterator import SameInstanceTypeFrameworkStackLenIterator


In [462]:
if torch.cuda.is_available() and False:
    cuda_device = args.cuda_device
    model = Resolver(
        cuda_device=cuda_device,
        vocab=vocab,
        field_type2embedder=field_type2embedder,
        field_type2seq2vec_encoder=field_type2seq2vec_encoder,
        field_type2seq2seq_encoder=field_type2seq2seq_encoder,
        action_classifier_feedforward=action_classifier_feedforward,
        action_num_pop_classifier_feedforward=action_num_pop_classifier_feedforward,
        framework2field_type2feedforward=framework2field_type2feedforward,
    )
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
    model = Resolver(
        cuda_device=cuda_device,
        vocab=vocab,
        field_type2embedder=field_type2embedder,
        field_type2seq2vec_encoder=field_type2seq2vec_encoder,
        field_type2seq2seq_encoder=field_type2seq2seq_encoder,
        action_classifier_feedforward=action_classifier_feedforward,
        action_num_pop_classifier_feedforward=action_num_pop_classifier_feedforward,
        framework2field_type2feedforward=framework2field_type2feedforward,
    )

# iterator = SameInstanceTypeFrameworkIterator(
#     shuffle=True,
#     batch_size=100, 
#     sorting_keys=[("token_node_resolveds", "num_tokens")],
# )
iterator = SameInstanceTypeFrameworkStackLenIterator(
    shuffle=True,
    batch_size=100, 
    sorting_keys=[("token_node_resolveds", "num_tokens")],
)
iterator.index_with(vocab)

optimizer = optim.SGD(model.parameters(), lr=0.1)

INFO     [allennlp.nn.initializers:293] Initializing parameters
INFO     [allennlp.nn.initializers:309] Done initializing parameters; the following parameters are using their default initialization from their code
INFO     [allennlp.nn.initializers:314]    action_classifier_feedforward._linear_layers.0.bias
INFO     [allennlp.nn.initializers:314]    action_classifier_feedforward._linear_layers.0.weight
INFO     [allennlp.nn.initializers:314]    action_classifier_feedforward._linear_layers.1.bias
INFO     [allennlp.nn.initializers:314]    action_classifier_feedforward._linear_layers.1.weight
INFO     [allennlp.nn.initializers:314]    action_num_pop_classifier_feedforward._linear_layers.0.bias
INFO     [allennlp.nn.initializers:314]    action_num_pop_classifier_feedforward._linear_layers.0.weight
INFO     [allennlp.nn.initializers:314]    action_num_pop_classifier_feedforward._linear_layers.1.bias
INFO     [allennlp.nn.initializers:314]    action_num_pop_classifier_feedforward._linear_la

INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_pos._module.weight_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_pos._module.weight_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_pos._module.weight_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.bias_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.bias_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.bias_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.weight_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.weight_hh_l0_reverse

INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.parse_curr_word._module.bias_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.parse_curr_word._module.bias_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.parse_curr_word._module.bias_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.parse_curr_word._module.weight_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.parse_curr_word._module.weight_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.parse_curr_word._module.weight_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.parse_curr_word._module.weight_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.bias_hh_l0_reverse
INFO   

INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_pos._module.weight_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_pos._module.weight_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_pos._module.weight_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_pos._module.weight_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.bias_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.bias_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.bias_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.weight_hh_l0

In [463]:
model.vocab.get_token_from_index(0, namespace='action_type_labels')

'RESOLVE'

In [464]:
cuda_device

-1

In [465]:
model.resolve_tensor

tensor(1)

In [466]:
# list(model.named_parameters())

In [467]:
serialization_dir = os.path.join(args.project_root, args.allennlp_serialization_dir_template.format(framework_names))

In [468]:
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=test_dataset,
#     train_dataset=train_dataset,
#     validation_dataset=train_dataset,
    patience=10,
    num_epochs=20,
    cuda_device=cuda_device,
    histogram_interval=100,
    serialization_dir=serialization_dir,
)

In [469]:
action_logits = torch.tensor([[-2.2126,  2.6022, -1.1655],
        [ 4.7340, -1.9992, -3.4521],
        [-1.9665,  2.4100, -1.2047],
        [-2.1353,  2.4847, -1.1260],
        [ 4.7492, -2.0234, -3.4460],
        [-1.4369,  1.9822, -1.2885],
        [ 1.0337,  0.3599, -2.0420],
        [ 5.0974, -2.3380, -3.4647],
        [ 5.4187, -2.4720, -3.6469],
        [-3.4045,  2.4903,  0.0773],
        [ 0.6384,  0.6764, -1.9942],
        [-2.2904,  2.7170, -1.2016],
        [ 4.6333, -1.9474, -3.4113],
        [-2.0811,  2.5367, -1.2174],
        [ 5.1840, -2.7536, -3.1499],
        [ 4.7421, -2.0138, -3.4485],
        [ 5.1121, -2.2290, -3.5999],
        [ 0.1843,  0.8324, -1.6990],
        [ 5.3854, -2.4593, -3.6309],
        [ 0.0324,  0.6715, -1.4173]])
action_logits = _cuda(action_logits, cuda_device)

action_type = torch.tensor([0, 2, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0])
action_type = _cuda(action_type, cuda_device)

In [470]:
action_probs, action_preds = action_logits.max(1)
action_resolve_preds = action_preds.eq_(model.resolve_tensor)

In [471]:
iter([1, 2, 3, 4])

In [472]:
defaultdict(lambda: defaultdict(dict))

defaultdict(<function __main__.<lambda>()>, {})

In [473]:
(action_resolve_preds, action_type, action_resolve_preds.eq(action_type))

(tensor([1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]),
 tensor([0, 2, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0]),
 tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
        dtype=torch.uint8))

In [474]:
action_resolve_preds.unsqueeze(-1).float() * action_logits

tensor([[-2.2126,  2.6022, -1.1655],
        [ 0.0000, -0.0000, -0.0000],
        [-1.9665,  2.4100, -1.2047],
        [-2.1353,  2.4847, -1.1260],
        [ 0.0000, -0.0000, -0.0000],
        [-1.4369,  1.9822, -1.2885],
        [ 0.0000,  0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000],
        [-3.4045,  2.4903,  0.0773],
        [ 0.6384,  0.6764, -1.9942],
        [-2.2904,  2.7170, -1.2016],
        [ 0.0000, -0.0000, -0.0000],
        [-2.0811,  2.5367, -1.2174],
        [ 0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000],
        [ 0.1843,  0.8324, -1.6990],
        [ 0.0000, -0.0000, -0.0000],
        [ 0.0324,  0.6715, -1.4173]])

In [475]:
embedded_fields = torch.ones(99, 62, 100)

In [476]:
embedded_fields.size()

torch.Size([99, 62, 100])

In [477]:
# embedded_fields

In [478]:
root_position_logits = torch.tensor([[0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1720, 0.1715, 0.1718],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1716, 0.1719, 0.1708],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1720, 0.1715, 0.1717],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1716, 0.1719, 0.1717],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1720, 0.1715, 0.1709]])

In [479]:
root_position_logits.shape
root_position = torch.tensor([ 2,  0, -1,  1,  0])
mask = root_position.eq(torch.tensor(-1)).unsqueeze(-1).expand_as(root_position_logits)
mask = root_position.gt(-1)
masked_root_position_logits = root_position_logits * mask.float().unsqueeze(-1).expand_as(root_position_logits)
loss = torch.nn.CrossEntropyLoss()
inp = root_position_logits
# crossEntropy = -torch.log(torch.gather(inp, 1, root_position.view(-1, 1)))
root_position
# root_position.eq(torch.tensor(-2)).expand_as(root_position_logits)

tensor([ 2,  0, -1,  1,  0])

In [480]:
masked_root_position_logits.shape

torch.Size([5, 31])

In [481]:
root_position.shape

torch.Size([5])

In [482]:
torch.ones(2, 3, 47).transpose(1, 2).shape

torch.Size([2, 47, 3])

In [483]:
[dim for dim in torch.tensor([[]]).size()]

[1, 0]

In [484]:
torch.tensor([[]]).shape[1]

0

In [485]:
model.field_type2embedder.keys()

odict_keys(['action', 'parse_curr_pos', 'parse_curr_word', 'parse_next_pos', 'parse_next_word', 'parse_prev_pos', 'parse_prev_word', 'pos', 'resolve_amr_pos', 'resolve_amr_word', 'resolve_dm_pos', 'resolve_dm_word', 'resolve_eds_pos', 'resolve_eds_word', 'resolve_psd_pos', 'resolve_psd_word', 'resolve_ucca_pos', 'resolve_ucca_word', 'resolved', 'template', 'token_node_label', 'token_node_prev_action', 'token_node_resolved', 'word'])

In [486]:
import random

In [489]:
random.randint(0, 0)

0

In [491]:
trainer.train()

INFO     [allennlp.training.trainer:465] Beginning training.
INFO     [allennlp.training.trainer:281] Epoch 0/19
INFO     [allennlp.training.trainer:283] Peak CPU memory usage MB: 20500.788
INFO     [allennlp.training.trainer:287] GPU 0 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 1 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 2 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 3 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 4 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 5 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 6 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 7 memory usage MB: 10
INFO     [allennlp.training.trainer:311] Training





















  0%|          | 0/2654 [00:00<?, ?it/s]




















action_type_accuracy: 0.5183, action_num_pop_accuracy: 0.2352, root_label_type_accuracy: 0.0000, child_edges_type_accuracy: 0.0000, loss: 4.62

action_type_accuracy: 0.5215, action_num_pop_accuracy: 0.2005, root_label_type_accuracy: 0.0000, child_edges_type_accuracy: 0.0000, loss: 3.1374 ||:   1%|          | 26/2654 [00:13<18:28,  2.37it/s]




















action_type_accuracy: 0.5215, action_num_pop_accuracy: 0.1994, root_label_type_accuracy: 0.0000, child_edges_type_accuracy: 0.0000, loss: 3.1348 ||:   1%|          | 27/2654 [00:13<19:21,  2.26it/s]




















action_type_accuracy: 0.5221, action_num_pop_accuracy: 0.1981, root_label_type_accuracy: 0.0000, child_edges_type_accuracy: 0.0000, loss: 3.1323 ||:   1%|          | 28/2654 [00:14<19:11,  2.28it/s]




















action_type_accuracy: 0.5218, action_num_pop_accuracy: 0.1971, root_label_type_accuracy: 0.0000, child_edges_type_accuracy: 0.0000, loss: 3.1310 ||:   1%|          | 29/2654 [00:14<19:24,  2.25it/s]




















action_type_accuracy: 0.5222, action_num_pop_accuracy: 0.1964, root_label_type_accuracy: 0.0000, child_edges_type_accuracy: 

KeyboardInterrupt: 

### Predictor

In [752]:
from mrp_library.predictors.mrp_predictor import MrpPredictor

In [753]:
reader = MRPDatasetActionReader()

In [754]:
mrp_predictor = MrpPredictor(model=model, dataset_reader=reader)

In [755]:
mrp_json = test_mrp_jsons[0]
cid = mrp_json.get('id')
doc = mrp_json.get('input')
frameworks = mrp_json.get('targets')
parse_json = test_parse_jsons[cid]

for framework in frameworks:
    companion_parser_states, companion_meta_data = mrp_json2parser_states(
        parse_json, 
        mrp_doc=doc,
        tokenized_parse_nodes=parse_json['nodes'],
    )

    data_instance = {
        'cid': cid,
        'doc': doc,
        'framework': framework,
        'mrp_json': mrp_json,
        'parse_json': parse_json,
        'companion_parser_states': companion_parser_states,
        'companion_meta_data': companion_meta_data,
    }

In [756]:
data_instance['framework']

'amr'

In [757]:
mrp_predictor.predict_json(data_instance)

WARNING  [mrp_library.predictors.mrp_predictor:92] ('num_node_unresolved', 0)
WARNING  [mrp_library.predictors.mrp_predictor:93] ('len(parse_token_stack)', 23)
WARNING  [mrp_library.predictors.mrp_predictor:94] ('token_stack', 0)
INFO     [mrp_library.predictors.mrp_predictor:270] ('curr_state', {'mode': 'predict', 'curr_node_id': 1, 'framework': 'amr', 'token_state': [(0, False, 'I', [])], 'parse_node_field_name2features': {'label': [i, was, very, much, worried, ,, for, it, was, becoming, clear, to, me, that, the, breakdown, of, my, plane, was, extremely, serious, .], 'lemma': [I, be, very, much, worry, ,, for, it, be, become, clear, to, I, that, the, breakdown, of, my, plane, be, extremely, serious, .], 'upos': [PRON, VERB, ADV, ADV, VERB, PUNCT, ADP, PRON, AUX, VERB, ADJ, ADP, PRON, SCONJ, DET, NOUN, ADP, PRON, NOUN, VERB, ADV, ADJ, PUNCT], 'xpos': [PRP, VBD, RB, RB, VBN, ,, IN, PRP, VBD, VBG, JJ, TO, PRP, IN, DT, NN, IN, PRP$, NN, VBD, RB, JJ, .]}, 'parse_node_state_field_name2feat

INFO     [mrp_library.predictors.mrp_predictor:270] ('curr_state', {'mode': 'predict', 'curr_node_id': 4, 'framework': 'amr', 'token_state': [(0, False, 'I', []), (1, False, 'was', []), (2, False, 'very', []), (3, False, 'much', [])], 'parse_node_field_name2features': {'label': [i, was, very, much, worried, ,, for, it, was, becoming, clear, to, me, that, the, breakdown, of, my, plane, was, extremely, serious, .], 'lemma': [I, be, very, much, worry, ,, for, it, be, become, clear, to, I, that, the, breakdown, of, my, plane, be, extremely, serious, .], 'upos': [PRON, VERB, ADV, ADV, VERB, PUNCT, ADP, PRON, AUX, VERB, ADJ, ADP, PRON, SCONJ, DET, NOUN, ADP, PRON, NOUN, VERB, ADV, ADJ, PUNCT], 'xpos': [PRP, VBD, RB, RB, VBN, ,, IN, PRP, VBD, VBG, JJ, TO, PRP, IN, DT, NN, IN, PRP$, NN, VBD, RB, JJ, .]}, 'parse_node_state_field_name2features': {'curr_label': [worried], 'curr_lemma': [worry], 'curr_upos': [VERB], 'curr_xpos': [VBN], 'prev_label': [i, was, very, much], 'prev_lemma': [I, be, very

INFO     [mrp_library.predictors.mrp_predictor:271] ('token_stack',
 [(0, False, 'I', []),
  (1, False, 'was', []),
  (2, False, 'very', []),
  (3, False, 'much', []),
  (4, False, 'worried', []),
  (5, False, ',', [])])
WARNING  [mrp_library.predictors.mrp_predictor:92] ('num_node_unresolved', 6)
WARNING  [mrp_library.predictors.mrp_predictor:93] ('len(parse_token_stack)', 17)
WARNING  [mrp_library.predictors.mrp_predictor:94] ('token_stack', 6)
INFO     [mrp_library.predictors.mrp_predictor:284] ('_json_to_instance: curr_state', {'mode': 'predict', 'curr_node_id': 6, 'framework': 'amr', 'token_state': [(0, False, 'I', []), (1, False, 'was', []), (2, False, 'very', []), (3, False, 'much', []), (4, False, 'worried', []), (5, False, ',', [])], 'parse_node_field_name2features': {'label': [i, was, very, much, worried, ,, for, it, was, becoming, clear, to, me, that, the, breakdown, of, my, plane, was, extremely, serious, .], 'lemma': [I, be, very, much, worry, ,, for, it, be, become, clear

INFO     [mrp_library.predictors.mrp_predictor:270] ('curr_state', {'mode': 'predict', 'curr_node_id': 9, 'framework': 'amr', 'token_state': [(0, False, 'I', []), (1, False, 'was', []), (2, False, 'very', []), (3, False, 'much', []), (4, False, 'worried', []), (5, False, ',', []), (6, False, 'for', []), (7, False, 'it', []), (8, False, 'was', [])], 'parse_node_field_name2features': {'label': [i, was, very, much, worried, ,, for, it, was, becoming, clear, to, me, that, the, breakdown, of, my, plane, was, extremely, serious, .], 'lemma': [I, be, very, much, worry, ,, for, it, be, become, clear, to, I, that, the, breakdown, of, my, plane, be, extremely, serious, .], 'upos': [PRON, VERB, ADV, ADV, VERB, PUNCT, ADP, PRON, AUX, VERB, ADJ, ADP, PRON, SCONJ, DET, NOUN, ADP, PRON, NOUN, VERB, ADV, ADJ, PUNCT], 'xpos': [PRP, VBD, RB, RB, VBN, ,, IN, PRP, VBD, VBG, JJ, TO, PRP, IN, DT, NN, IN, PRP$, NN, VBD, RB, JJ, .]}, 'parse_node_state_field_name2features': {'curr_label': [becoming], 'curr_lem

INFO     [mrp_library.predictors.mrp_predictor:270] ('curr_state', {'mode': 'predict', 'curr_node_id': 11, 'framework': 'amr', 'token_state': [(0, False, 'I', []), (1, False, 'was', []), (2, False, 'very', []), (3, False, 'much', []), (4, False, 'worried', []), (5, False, ',', []), (6, False, 'for', []), (7, False, 'it', []), (8, False, 'was', []), (9, False, 'becoming', []), (10, False, 'clear', [])], 'parse_node_field_name2features': {'label': [i, was, very, much, worried, ,, for, it, was, becoming, clear, to, me, that, the, breakdown, of, my, plane, was, extremely, serious, .], 'lemma': [I, be, very, much, worry, ,, for, it, be, become, clear, to, I, that, the, breakdown, of, my, plane, be, extremely, serious, .], 'upos': [PRON, VERB, ADV, ADV, VERB, PUNCT, ADP, PRON, AUX, VERB, ADJ, ADP, PRON, SCONJ, DET, NOUN, ADP, PRON, NOUN, VERB, ADV, ADJ, PUNCT], 'xpos': [PRP, VBD, RB, RB, VBN, ,, IN, PRP, VBD, VBG, JJ, TO, PRP, IN, DT, NN, IN, PRP$, NN, VBD, RB, JJ, .]}, 'parse_node_state_fie

...

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:253] ('token_node_id', 2, True)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:258] ('action_num_pop', 2)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:260] ('resolved_child_parse_node_ids', [2])
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:262] ('resolved_root_parse_node_id', 1)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:305] ('resolved_feature_size', 1)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:307] ('resolved_root_field_name2features', {'parse_node_label': [was], 'parse_node_lemma': [be], 'parse_node_upos': [VERB], 'parse_node_xpos': [VBD]})
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:310] ('resolved_child_field_name2features', {'parse_node_label': [very], 'parse_node_lemma': [very], 'parse_node_upos': [ADV], 'parse_node_xpos': [RB]})
INFO     [mrp_library.predictors.mrp_predictor:284] ('_json_to_instance: curr_state', {'mode': 'predict', 'curr_node_id': 23, '

INFO     [mrp_library.predictors.mrp_predictor:271] ('token_stack',
 [(0, False, 'I', []),
  (1,
   True,
   'was',
   [(1, False, 'was', []),
    (2,
     True,
     'very',
     [(2, False, 'very', []),
      (3,
       True,
       'much',
       [(3, False, 'much', []),
        (4,
         True,
         'worried',
         [(4, False, 'worried', []),
          (5,
           True,
           ',',
           [(5, False, ',', []),
            (6,
             True,
             'for',
             [(6, False, 'for', []),
              (7,
               True,
               'it',
               [(7, False, 'it', []),
                (8,
                 True,
                 'was',
                 [(8, False, 'was', []),
                  (9,
                   True,
                   'becoming',
                   [(9, False, 'becoming', []),
                    (10,
                     True,
                     'clear',
                     [(10, False, 'clear', []),
       

WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:253] ('token_node_id', 0, False)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:253] ('token_node_id', 1, True)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:258] ('action_num_pop', 2)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:260] ('resolved_child_parse_node_ids', [1])
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:262] ('resolved_root_parse_node_id', 0)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:305] ('resolved_feature_size', 1)
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:307] ('resolved_root_field_name2features', {'parse_node_label': [i], 'parse_node_lemma': [I], 'parse_node_upos': [PRON], 'parse_node_xpos': [PRP]})
WARNING  [mrp_library.dataset_readers.mrp_jsons_actions:310] ('resolved_child_field_name2features', {'parse_node_label': [was], 'parse_node_lemma': [be], 'parse_node_upos': [VERB], 'parse_node_xpos': [VBD]})
INFO     [mrp_library.predictors.mrp_predi

INFO     [mrp_library.predictors.mrp_predictor:271] ('token_stack',
 [(0,
   True,
   'I',
   [(0, False, 'I', []),
    (1,
     True,
     'was',
     [(1, False, 'was', []),
      (2,
       True,
       'very',
       [(2, False, 'very', []),
        (3,
         True,
         'much',
         [(3, False, 'much', []),
          (4,
           True,
           'worried',
           [(4, False, 'worried', []),
            (5,
             True,
             ',',
             [(5, False, ',', []),
              (6,
               True,
               'for',
               [(6, False, 'for', []),
                (7,
                 True,
                 'it',
                 [(7, False, 'it', []),
                  (8,
                   True,
                   'was',
                   [(8, False, 'was', []),
                    (9,
                     True,
                     'becoming',
                     [(9, False, 'becoming', []),
                      (10,
             

{}

In [674]:
model.framework2field_type2feedforward['eds']['child_edges']

FeedForward(
  (_linear_layers): ModuleList(
    (0): Linear(in_features=400, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=0, bias=True)
  )
  (_dropout): ModuleList(
    (0): Dropout(p=0.0)
    (1): Dropout(p=0.0)
  )
)

In [ ]:
[1, 2, 3][-3]

In [ ]:
mrp_json

In [ ]:
torch.ones(98, 29, 5).squeeze(-1)

In [ ]:
torch.ones(98, 29, 5).view(-1, 29).shape

In [ ]:
vocab.get_token_index()

In [ ]:
vocab.get_token_from_index(2, namespace='action_type')

In [ ]:
vocab.get_token_from_index(1, namespace='resolve_label_root_label')

In [ ]:
token_state = [[26, True, 'H', [[23, True, 'A', [[0, True, 'E', [[0, False, 'The', []]]], [1, True, 'C', [[1, False, 'Lakers', []]]]]], [2, True, 'P', [[2, False, 'advanced', []]]], [25, True, 'A', [[3, True, 'R', [[3, False, 'through', []]]], [4, True, 'E', [[4, False, 'the', []]]], [24, True, 'P', [[5, True, 'T', [[5, False, '1982', []]]], [6, True, 'C', [[6, False, 'playoffs', []]]]]]]]]], [7, True, 'L', [[7, False, 'and', []]]], [8, True, 'P', [[8, False, 'faced', []]]], [9, True, 'A', [[9, False, 'Philadelphia', []]]], [27, True, 'D', [[10, True, 'R', [[10, False, 'for', []]]], [11, True, 'E', [[11, False, 'the', []]]], [12, True, 'Q', [[12, False, 'second', []]]], [13, True, 'C', [[13, False, 'time', []]]]]], [14, True, 'R', [[14, False, 'in', []]]], [15, True, 'Q', [[15, False, 'three', []]]], [16, False, 'years', []]]

In [ ]:
pprint.pprint(token_state)

In [ ]:
vocab.get_token_from_index(0, namespace='labels')

In [ ]:
vocab.get_token_index('RESOLVE')

In [ ]:
vocab.get_token_from_index(3, namespace='resolved')

In [ ]:
vocab.get_token_from_index(5, namespace='token_node_prev_action')